In [55]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix


## Movielens full - EDA
a) Gör en EDA för att förstå datasetet. Inkludera olika slags plots. Begränsa dig inte till frågorna nedan,
utan försök undersöka fler aspekter av datan.

* Jag börjar med att undersöka vad datasetet har för columner med df.head och df.info()
* Samt så har jag läst på "Read me" filen som tillhör datasetet

In [57]:
data= 'movies.csv'
names=
df_movies= pd.read_csv("Data/ml-latest/movies.csv", index_col = 0)


dataset=pd.DataFrame(data=data.data,columns=names)
dataset['class']=data.target

dataset


SyntaxError: invalid syntax (3813273917.py, line 2)

In [ ]:
ratings = 'ratings.csv'
df_ratings = pd.read_csv("Data/ml-latest/ratings.csv", index_col = 0)

df_ratings.head()

,movieId,rating,timestamp
userId,,,
1,1,4.0,1225734739
1,110,4.0,1225865086
1,158,4.0,1225733503
1,260,4.5,1225735204
1,356,5.0,1225735119


In [ ]:
# Jag tar df.info 
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86537 entries, 1 to 288983
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   86537 non-null  object
 1   genres  86537 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33832162 entries, 1 to 330975
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   rating     float64
 2   timestamp  int64  
dtypes: float64(1), int64(2)
memory usage: 1.0 GB


b) Vilka är de 10 filmerna med flest ratings?

In [ ]:

df_ratings = pd.read_csv(ratings, usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})


FileNotFoundError: [Errno 2] No such file or directory: 'movies.csv'





c) Beräkna den genomsnittliga ratingen för dessa 10 filmerna med flest ratings.

d) Gör en plot över årtal och antalet filmer representerade i datasetet.

e) Gör en plot över antalet ratings mot movieId.

f) Beräkna genomsnittliga ratings för de top 10 filmerna med flest ratings. Gör ett stapeldiagram över
dessa.

1.2 Skapa gles matris

Likt i videon i uppgift 1.0 skapade du en pivottabell av dataframet med index: "movieId", columns: "userId"
och values: "ratings". Denna pivottabell är dock "dyr" att skapa och förmodligen kommer inte din dator att
klara av skapa den om du inte filtrerar bort viss data. Fundera ut ett lämpligt sätt att filtrera ditt dataset,
pröva dig fram och motivera.#

Skapa en gles (sparse) matris av denna pivottabell mha scipy.sparse.csc_matrix(). Vill du använda
dig av all data går det också att lösa, men du behöver lösa hur du skapar den glesa matrisen utan pandas
pivot-tabell.

 1.3 Recommender system

Skapa ett recommender system med KNN och låt systemet ta input från användaren och skriva ut top 5
rekommenderade filmerna, baserat på användarens sökquery. Observera att det finns ett logiskt fel i
videon som gör att rekommendationerna inte blir så bra, försök hitta felet och åtgärda det.

a) Beskriv med ord hur ditt system fungerar.

b) Leta online och läs vidare om rekommenderarsystem och beskriv kort hur dem fungerar. Glöm inte
källhänvisa.

